In [ ]:
try:
    reload_flag + 1
except:
    %load_ext autoreload
    %autoreload 2
    reload_flag = 1

from hwacctools.comp_graph import cgraph, cnodes, core
import onnx
import matplotlib.pyplot as plt
import rectpack
import seaborn as sns
import pandas as pd

nx_model = onnx.load("onnx_models/mbv2_cifar10_int8_binary.onnx")

In [ ]:
packer_dict = {
    "MR-OFF-BSSF-BFF-SA": None,
    "MR-OFF-BSSF-BBF-SO": rectpack.newPacker(
        mode=rectpack.PackingMode.Offline,
        bin_algo=rectpack.PackingBin.BBF,
        rotation=False,
        pack_algo=rectpack.MaxRectsBssf,
        sort_algo=rectpack.SORT_NONE,
    ),
    "MR-ON-BSSF-BNF-SO": rectpack.newPacker(
        mode=rectpack.PackingMode.Online,
        bin_algo=rectpack.PackingBin.BNF,
        rotation=False,
        pack_algo=rectpack.MaxRectsBssf,
        sort_algo=rectpack.SORT_NONE,
    ),
    "SKYBLWM-ON-BNF-SO": rectpack.newPacker(
        mode=rectpack.PackingMode.Online,
        rotation=False,
        bin_algo=rectpack.PackingBin.BNF,
        pack_algo=rectpack.SkylineBlWm,
        sort_algo=rectpack.SORT_NONE,
    ),
    "MR-ON-BSSF-BBF-SO": rectpack.newPacker(
        mode=rectpack.PackingMode.Online,
        bin_algo=rectpack.PackingBin.BBF,
        rotation=False,
        pack_algo=rectpack.MaxRectsBssf,
        sort_algo=rectpack.SORT_NONE,
    ),
    "SKYBLWM-ON-BBF-SO": rectpack.newPacker(
        mode=rectpack.PackingMode.Online,
        bin_algo=rectpack.PackingBin.BBF,
        rotation=False,
        pack_algo=rectpack.SkylineBlWm,
        sort_algo=rectpack.SORT_NONE,
    ),
}

# packed_models = {set_name: core.packed_model.from_onnx_model(nx_model, (256,256), packer=packer) for set_name, packer in packer_dict.items()}

num_subplot_rows = len(packer_dict) // 2 + len(packer_dict) % 2

packed_models = {set_name: core.packed_model.from_onnx_model(nx_model, (256,256), packer=packer) for set_name, packer in packer_dict.items()}

In [ ]:
fig, axs = plt.subplots(2,num_subplot_rows,figsize=(10, 4), dpi=300)
for i,(set_name,packed_model) in enumerate(packed_models.items()):
    model = packed_model
    sim = core.QRAccModel(model)
    sim.plot_write_lineplot(name = set_name, ax=axs[i//num_subplot_rows, i%num_subplot_rows],)
plt.tight_layout()
# set all ax limits to 60 y and 90 x
# for ax in axs.flat:
#     ax.set_xlim(0, 90)
#     ax.set_ylim(0, 53)

In [ ]:
fig, axs = plt.subplots(2,num_subplot_rows,figsize=(10, 4), dpi=300)
for i,(set_name,packed_model) in enumerate(packed_models.items()):
    model = packed_model
    sim = core.QRAccModel(model)
    sim.plot_bin_writes_bargraph(name = set_name, ax=axs[i//num_subplot_rows, i%num_subplot_rows],)
plt.tight_layout()
# set all ax limits to 60 y and 90 x
# for ax in axs.flat:
#     ax.set_xlim(0, 90)
#     ax.set_ylim(0, 53)

In [ ]:
# Experiments on all relevant packers

pack_alg = {
    'BL'    :rectpack.MaxRectsBl,
    'BSSF'  :rectpack.MaxRectsBssf,
    'BAF'   :rectpack.MaxRectsBaf,
}
mode = {
    'ON'    :rectpack.PackingMode.Online,
    'OFF'   :rectpack.PackingMode.Offline,
}
sort_alg = { # Ratio and Sside are the best-performing sorts
    'SO'    :rectpack.SORT_NONE, 
    'SF'    :rectpack.SORT_RATIO,
    'SS'    :rectpack.SORT_SSIDE,
}
bin_alg = {
    'BNF'   :rectpack.PackingBin.BNF,
    'BBF'   :rectpack.PackingBin.BBF,
    'BFF'   :rectpack.PackingBin.BBF,
}

packer_dict = {
    f"MR-{m}-{p}-{b}-{s}": rectpack.newPacker(
        mode=mode[m],
        bin_algo=bin_alg[b],
        rotation=False,
        pack_algo=pack_alg[p],
        sort_algo=sort_alg[s],
    ) for m in mode for p in pack_alg for b in bin_alg for s in sort_alg
}

raw_sim_df = pd.DataFrame(columns=['set_name','total_bins', 'total_bin_writes', 'utilization'])

In [ ]:
for set_name, packer in packer_dict.items():
    packed_model = core.packed_model.from_onnx_model(nx_model, (256,256), packer=packer)
    sim = core.QRAccModel(packed_model)
    raw_sim_df = pd.concat([raw_sim_df, pd.DataFrame({
        'set_name': [set_name],
        'total_bins': [sim.total_bins],
        'total_bin_writes': [sim.weight_bin_writes],
        'utilization': [sim.utilization],
    })], ignore_index=True)
# separate sim_df set_name into columns for mode, pack_algo, bin_algo, sort_algo by splitting between -
sim_df = raw_sim_df.copy()
sim_df[['heuristic','mode', 'pack_algo', 'bin_algo', 'sort_algo']] = sim_df['set_name'].str.split('-', expand=True)
# convert mode, pack_algo, bin_algo, sort_algo to categorical
sim_df['mode'] = sim_df['mode'].astype('category')
sim_df['pack_algo'] = sim_df['pack_algo'].astype('category')
sim_df['bin_algo'] = sim_df['bin_algo'].astype('category')
sim_df['sort_algo'] = sim_df['sort_algo'].astype('category')
# convert total_bins and total_bin_writes to int
sim_df['total_bins'] = sim_df['total_bins'].astype(int)
sim_df['total_bin_writes'] = sim_df['total_bin_writes'].astype(int)
# convert utilization to float
sim_df['utilization'] = sim_df['utilization'].astype(float)
sim_df

In [ ]:
fig, axs = plt.subplots(figsize=(3, 3), dpi=300)
# move legend outside
sns.scatterplot(data=sim_df, x='utilization', y='total_bin_writes', hue='bin_algo', style='pack_algo', ax=axs, s=100)
axs.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
# rename x and y
axs.set_xlabel('Utilization')
axs.set_ylabel('$N_{writes}$')

In [ ]:
fig, axs = plt.subplots(figsize=(3, 3), dpi=300)
# move legend outside
sns.scatterplot(data=sim_df, x='total_bins', y='total_bin_writes', hue='bin_algo', style='pack_algo', ax=axs)
axs.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
# rename x and y
axs.set_xlabel('$N_{bins}$')
axs.set_ylabel('$N_{writes}$')

In [ ]:
# plot sim_df as a barplot with seaborn
sns.set_theme(style="whitegrid")
# without fills
a = sns.relplot(data=sim_df, x='total_bins', y='total_bin_writes', hue='bin_algo', col='pack_algo', size='mode', kind='scatter', style = 'sort_algo', legend='brief', s=100)
# set figsize
# a.fig.set_size_inches(5,3)
a.fig.set_size_inches(5.5, 2.5)
a.fig.set_dpi(300)
# set legend outside
a._legend.set_bbox_to_anchor((1.2, 0.5))
# set anchor to upper right
a.set_axis_labels("$N_{bins}$", "$N_{writes}$")
# add padding vertically
a.fig.tight_layout()


In [ ]:
# Try the best offline packer

nx_model = onnx.load("onnx_models/mbv2_cifar10_int8_binary.onnx")
packer = rectpack.newPacker(
    mode=rectpack.PackingMode.Offline,
    bin_algo=rectpack.PackingBin.BBF,
    rotation=False,
    pack_algo=rectpack.MaxRectsBssf,
    sort_algo=rectpack.SORT_AREA,
)
packed_model = core.packed_model.from_onnx_model(nx_model, (256,256), packer=packer)
sim = core.QRAccModel(packed_model, num_cores=2)

fig, ax = plt.subplots(1, 1, figsize=(5, 3), dpi=300)

sim.plot_write_lineplot(name = 'MR-OFF-BSSF-BBF-SA', ax=ax)

In [ ]:
results_df = pd.DataFrame.from_dict(results, orient='index')
# turn index into set_name and ncore column
results_df.reset_index(inplace=True)
results_df.rename(columns={'index': 'set_name'}, inplace=True)
# rename level_0 to set_name and level_1 to ncore
results_df.rename(columns={'level_0': 'set_name', 'level_1': 'ncore'}, inplace=True)
results_df

In [ ]:
# Try the best offline packer

nx_model = onnx.load("onnx_models/mbv2_cifar10_int8_binary.onnx")
packers = {
    "MR-OFF-BSSF-BBF-SA"    : rectpack.newPacker(
        mode=rectpack.PackingMode.Offline,
        bin_algo=rectpack.PackingBin.BBF,
        rotation=False,
        pack_algo=rectpack.MaxRectsBssf,
        sort_algo=rectpack.SORT_AREA,
    ),
    "MR-OFF-BSSF-BNF-SO"    : rectpack.newPacker(
        mode=rectpack.PackingMode.Offline,
        bin_algo=rectpack.PackingBin.BNF,
        rotation=False,
        pack_algo=rectpack.MaxRectsBssf,
        sort_algo=rectpack.SORT_NONE,
    ),
    "MR-OFF-BSSF-BBF-SO"    : rectpack.newPacker(
        mode=rectpack.PackingMode.Offline,
        bin_algo=rectpack.PackingBin.BBF,
        rotation=False,
        pack_algo=rectpack.MaxRectsBssf,
        sort_algo=rectpack.SORT_NONE,
    ),
}
# packed_model = core.packed_model.from_onnx_model(nx_model, (256,256), packer=packer)

ncores = [1, 2, 4, 8, 16]

results = {}

for set_name, packer in packers.items():
    packed_model = core.packed_model.from_onnx_model(nx_model, (256,256), packer=packer)
    print(f"Running simulation with {set_name}")
    
    for ncore in ncores:
        print(f"Running simulation with {ncore} cores")
        sim = core.QRAccModel(packed_model, num_cores=ncore)
        results[(set_name, ncore)] = {
            'utilization': sim.utilization,
            'total_bins': sim.total_bins,
            'total_bin_writes': sim.weight_bin_writes,
        }
df_results = pd.DataFrame.from_dict(results, orient='index')
# turn index into set_name and ncore column
df_results.reset_index(inplace=True)
df_results.rename(columns={'index': 'set_name'}, inplace=True)
# rename level_0 to set_name and level_1 to ncore
df_results.rename(columns={'level_0': 'set_name', 'level_1': 'ncore'}, inplace=True)
df_results[['heuristic','mode', 'pack_algo', 'bin_algo', 'sort_algo']] = df_results['set_name'].str.split('-', expand=True)
# convert mode, pack_algo, bin_algo, sort_algo to categorical
df_results['mode'] = df_results['mode'].astype('category')
df_results['pack_algo'] = df_results['pack_algo'].astype('category')
df_results['bin_algo'] = df_results['bin_algo'].astype('category')
df_results['sort_algo'] = df_results['sort_algo'].astype('category')
# convert total_bins and total_bin_writes to int
df_results['total_bins'] = df_results['total_bins'].astype(int)
df_results['total_bin_writes'] = df_results['total_bin_writes'].astype(int)
df_results['utilization'] = df_results['utilization'].astype(float)
df_results.to_csv('packing_vs_ncores.csv', index=False)

In [ ]:
df_results 

In [ ]:
df_results = pd.read_csv('packing_vs_ncores.csv')
# set ncore as int
plt.figure(figsize=(2, 2), dpi=300)
df_results['ncore'] = df_results['ncore'].astype(int)
sns.set_theme(style="whitegrid")
# Plot the results as a barplot with seaborn
a = sns.lineplot(data=df_results, x='ncore', y='total_bin_writes', hue='set_name', marker='o')
# put legend outside
a.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
# turn x labels vertical
# a.set_xticklabels(a.get_xticklabels(), rotation=90)
plt.title("LFU Experiments")
# set xtixks to 1, 2, 4, 8, 16
plt.xticks([1, 2, 4, 8, 16])
plt.xlabel("# Cores")
plt.ylabel("# Writes")